- This is an experimental eval strat with need for human (me) vaidation. The idead is to have models do TTS then compare this with the actual text of an uadio file
- Afterwords, the actual and TTX text are compared using cosine similarity and then the an average is obtained (currently testing for 100 audio samples)
- The model with the highets is best. 
- PS this is just an idead for the Swahili TTS. Exisitng eval methods for common langs ('English, etc'), are already great.

----


### Data Prep:

- Data used: mozilla-foundation/common_voice_11_0
- [Source](https://huggingface.co/datasets/mozilla-foundation/common_voice_11_0/viewer/sw)

In [1]:
from datasets import load_dataset

In [2]:
# all 66 books
#dataset = load_dataset("bookbot/OpenBible_Swahili", trust_remote_code=True)
# all 66 books, clean subset
#dataset = load_dataset("bookbot/OpenBible_Swahili", "clean", trust_remote_code=True)
# only Genesis
#dataset = load_dataset("bookbot/OpenBible_Swahili", "GEN", trust_remote_code=True)
# only Genesis, clean subset
#dataset = load_dataset("bookbot/OpenBible_Swahili", "GEN_clean", trust_remote_code=True)


In [4]:
ds = load_dataset("mozilla-foundation/common_voice_11_0", "sw", split="test",trust_remote_code=True)

In [7]:
ds

Dataset({
    features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
    num_rows: 10238
})

----

In [15]:
import pandas as pd

In [16]:
df = pd.DataFrame(ds)
df.head()

,client_id,path,audio,sentence,up_votes,down_votes,age,gender,accent,locale,segment
0,0133d8ddf5c1a3c678fde017e0b07d2835bfd707d5b3ec...,C:\Users\Ara\.cache\huggingface\datasets\downl...,{'path': 'C:\Users\Ara\.cache\huggingface\data...,wachambuzi wa soka wanamtaja Messi kama nyota ...,2,0,twenties,female,,sw,
1,01c95772efd3fbe4a1122206c7474c77ed6591c8c9fb00...,C:\Users\Ara\.cache\huggingface\datasets\downl...,{'path': 'C:\Users\Ara\.cache\huggingface\data...,romario aliingia kwenye orodha ya wachezaji wa...,2,1,,,,sw,
2,023711185d4404ff398c2697f2e72868d1ecf69a92b581...,C:\Users\Ara\.cache\huggingface\datasets\downl...,{'path': 'C:\Users\Ara\.cache\huggingface\data...,Sote twesangaa twelipomuona mwalimu Ali apika,2,1,twenties,male,,sw,
3,0244639ffd7ec755a01b21ea204735ca3c44443e9cf46c...,C:\Users\Ara\.cache\huggingface\datasets\downl...,{'path': 'C:\Users\Ara\.cache\huggingface\data...,Inajulikana kama shina la Warangi.,2,0,,,,sw,
4,260a28cb0a4ad26c1e5c43305877ed565bdac47119963e...,C:\Users\Ara\.cache\huggingface\datasets\downl...,{'path': 'C:\Users\Ara\.cache\huggingface\data...,Mwidhi aliyejepa simu amekatiwa kifungo cha mw...,2,1,,,,sw,


In [17]:
print(df.shape)

(10238, 11)


In [18]:
df['accent'].value_counts()

accent
                                                                                                                                                                                                                                           9439
Fluent swahili                                                                                                                                                                                                                              208
native                                                                                                                                                                                                                                       97
Fluent,Fluent swahili                                                                                                                                                                                                                        85
Fluent accent                    

In [22]:
df['age'].value_counts()

age
twenties    5579
            3461
thirties     700
fourties     293
teens        121
fifties       84
Name: count, dtype: int64

In [23]:
df = df.drop(columns=['audio', 'locale', 'segment', 'client_id', 'locale'])

In [24]:
df.columns

Index(['path', 'sentence', 'up_votes', 'down_votes', 'age', 'gender',
       'accent'],
      dtype='object')

In [25]:
df.to_csv('mozilla_mp3files.csv', index=False)

-----

### Sampling

In [28]:
import ipywidgets as widgets
from IPython.display import display, Audio
import random

In [29]:
def play_audio(file_path):
    audio_widget = Audio(file_path)
    display(audio_widget)

In [34]:
def sample():
    sample_index = random.randint(0, 10237)
    print('Text: ' + df['sentence'].iloc[sample_index])
    print('Age: ' + df['age'].iloc[sample_index])
    print('Gender: ' + df['gender'].iloc[sample_index])
    play_audio(df['path'].iloc[sample_index])
    

In [35]:
sample()

Text: Wakati mwingine kama fedha kwa ajili ya kununua na kuuza bidhaa
Age: twenties
Gender: female


----

### Evaluation